<a href="https://colab.research.google.com/github/Changvang/GoogleColab/blob/main/DemoDFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models

import gensim

# Load data set with 2150 Kaggle dataset

In [ ]:
import pandas as pd

# load data set with 2150 Kaggle dataset
dataset = pd.read_csv('drive/My Drive/Colab Notebooks/dataset/voted-kaggle-dataset.csv')
print(dataset.head())

In [ ]:
dataset.iloc[0:5,:]

In [ ]:
dataset.info()

# Data Preprocessing

In [ ]:
dataset.columns.values

In [ ]:
modified_dataset = dataset.loc[:, ['Title', 'Subtitle', 'Description']]

In [ ]:
modified_dataset.iloc[0:5,:]

In [ ]:
modified_dataset.info()

In [ ]:
modified_dataset.isnull()

In [ ]:
# some row will no data
print(modified_dataset.isnull().sum())
print("total null values: ", sum(modified_dataset.isnull().values.ravel()))
print("total number of rows containing null values: ", sum([True for idx,row in modified_dataset.iterrows() if any(row.isnull())]))

In [ ]:
#remove rows have null value
modified_dataset = modified_dataset.dropna()
modified_dataset.info()

In [ ]:
import string
import re



# Remove stropword, puncation and lower casing

In [ ]:
remove_digits = str.maketrans('', '' , string.digits)
# three arguments so all value in third argument will assign to None

In [ ]:
for column in ['Title', 'Subtitle','Description']:
  #lower all string
  modified_dataset[column] = modified_dataset[column].map(lambda x: x.lower())

In [ ]:
modified_dataset.iloc[0:3,:]

In [ ]:
exclude = '[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'

In [ ]:
for column in ['Title','Subtitle','Description']:
  # Delete digits
  modified_dataset[column] = modified_dataset[column].map(lambda x : x.translate(remove_digits))
  # Delete punctation
  modified_dataset[column] = modified_dataset[column].map(lambda x : re.sub(str(exclude), '', x))

In [ ]:
modified_dataset.iloc[0:3,:]


# Using stemming and lemmalization for preprocessing

In [ ]:
# Tag is a main title of this document
tag_dataset = dataset['Tags']
tag_dataset.isnull().sum()
#tag_dataset=tag_dataset.dropna()
tag_dataset.isnull().sum()

In [ ]:
print(len(tag_dataset))

In [ ]:
def convert(lst): 
    return (lst.split("\n"))

In [ ]:
unique_tag = []
for i in range(len(tag_dataset)):
  tag_string = str(tag_dataset[i])
  if tag_string != "nan" :
    tag_word=convert(tag_string)
    for j in range(len(tag_word)):
      if tag_word[j] not in unique_tag:
        unique_tag.append(tag_word[j])
print(len(unique_tag))

In [ ]:
import nltk
nltk.download('punkt')
tokenized_dataframe = modified_dataset.apply(lambda row: nltk.word_tokenize(row['Description']), axis = 1)
print(type(tokenized_dataframe))

In [ ]:
print(tokenized_dataframe[0:3])

In [ ]:
#check single word
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
def lemmatize_text(text):
    return  [ps.stem(w) for w in text if len(w)>5]

In [ ]:
stemmed_dataset = tokenized_dataframe.apply(lemmatize_text)

In [ ]:
stemmed_dataset[0:3]

#Exploratory Analysis

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

dataset_words=''.join(list(str(stemmed_dataset.values)))
print(type(dataset_words))
wordcloud = WordCloud(width = 800, height = 500, 
                background_color ='white',  
                min_font_size = 10).generate(dataset_words) 

plt.figure(figsize = (5, 5), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show()

# Create Dictionary and Corpus for LDA model

In [ ]:
dictionary_of_words =gensim.corpora.Dictionary(stemmed_dataset)
print(len(dictionary_of_words))

In [ ]:
print(dictionary_of_words.values)

In [ ]:
word_corpus = [dictionary_of_words.doc2bow(word) for word in stemmed_dataset]

In [ ]:
print(word_corpus[0])

# Training the first model

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=word_corpus,
    id2word=dictionary_of_words,
    num_topics=50,
    random_state=101,
    update_every=1,
    chunksize=300,
    passes=50,
    alpha='auto',
    per_word_topics=True
    )

In [ ]:
#print(lda_model.print_topics(-1))

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

coherence_val = CoherenceModel(model=lda_model, texts=stemmed_dataset, dictionary=dictionary_of_words, coherence='c_v').get_coherence()

print('Coherence Score: ', coherence_val)

In [ ]:
models = []
coherence_value = []
for topic_number in range(10,100,10):
  lda_model = gensim.models.ldamodel.LdaModel(corpus=word_corpus,
                                           id2word=dictionary_of_words,
                                           num_topics=topic_number, 
                                           random_state=101,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)
  models.append(lda_model)
  coherence_model_lda = CoherenceModel(model=lda_model, texts=stemmed_dataset, dictionary=dictionary_of_words, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  coherence_value.append(coherence_lda)
  print("number of topics ",topic_number,"coherence_value :" , coherence_lda)

In [ ]:
# from gensim.models.coherencemodel import CoherenceModel


# # Compute Coherence Score
# cohr_val = CoherenceModel(model=lda_model, texts=stemmed_dataset, dictionary=dictionary_of_words, coherence='c_v').get_coherence()

# print('\nCoherence Score: ', cohr_val)


# Support multi core genshin lda

In [ ]:
from gensim.test.utils import common_corpus, common_dictionary
lda_multicore_model = gensim.models.ldamulticore.LdaMulticore(corpus=word_corpus, 
                                                              num_topics=50, 
                                                              id2word=dictionary_of_words,                                                             
                                                              chunksize=100, 
                                                              passes=50,                                
                                                              alpha='symmetric',
                                                              eta=0.1,
                                                              decay=0.5, 
                                                              offset=1.0, 
                                                              gamma_threshold=0.001,
                                                              random_state=101,
                                                              minimum_probability=0.01,
                                                              minimum_phi_value=0.01,
                                                              per_word_topics=False)

In [ ]:
from gensim.models.coherencemodel import CoherenceModel


# Compute Coherence Score
cohr_lda_multicore_model1 = CoherenceModel(model=lda_multicore_model, texts=stemmed_dataset, dictionary=dictionary_of_words, coherence='c_v').get_coherence()

print('\nCoherence Score: ', cohr_lda_multicore_model1)

In [ ]:
# from gensim.test.utils import common_corpus, common_dictionary
# lda_multicore_model2 = gensim.models.ldamulticore.LdaMulticore(corpus=word_corpus, 
#                                                               num_topics=319, 
#                                                               id2word=dictionary_of_words,                                                             
#                                                               chunksize=100, 
#                                                               passes=50,                                
#                                                               alpha='symmetric',
#                                                               eta=0.1,
#                                                               decay=0.5, 
#                                                               offset=1.0, 
#                                                               gamma_threshold=0.001,
#                                                               random_state=101,
#                                                               minimum_probability=0.01,
#                                                               minimum_phi_value=0.01,
#                                                               per_word_topics=False)

In [ ]:
# from gensim.models.coherencemodel import CoherenceModel


# # Compute Coherence Score
# cohr_lda_multicore_model2 = CoherenceModel(model=lda_multicore_model2, texts=stemmed_dataset, dictionary=dictionary_of_words, coherence='c_v').get_coherence()

# print('\nCoherence Score: ', cohr_lda_multicore_model2)

# Find topic in a document

In [ ]:
v = lda_model[word_corpus[2]]
print(type(lda_model[word_corpus[2]]))
z=sorted(v[0], key=lambda tup: -1*tup[1])
print(z)
print(v[0])

In [ ]:
for  index,score in sorted(lda_model[word_corpus[2]][0], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

In [ ]:
topics = lda_model.show_topics(formatted=False)
topic_words = dict(topics[0][1])
wordcloud.generate_from_frequencies(topic_words, max_font_size=100)
plt.figure(figsize = (5, 5), facecolor = None) 
plt.title("topic 0")
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,15),frameon=0)
a = fig.add_subplot(1, 2, 1)
topic_words = dict(topics[1][1])
wordcloud.generate_from_frequencies(topic_words, max_font_size=100)
imgplot = plt.imshow(wordcloud)
a.set_title('topic 0')

a = fig.add_subplot(1, 2, 2)
topic_words = dict(topics[2][1])
wordcloud.generate_from_frequencies(topic_words, max_font_size=100)
imgplot = plt.imshow(wordcloud)

a.set_title('topic 1')